In [1]:
import pandas as pd
# Load the data from Excel files
bills_data = pd.read_excel('Pharmacy python test(1).xlsx', sheet_name = 'Merge')
database_data = pd.read_excel('Pharmacy python test(1).xlsx' ,sheet_name = 'Main_Data')

In [21]:
bills_data = bills_data.astype(str)
database_data = database_data.astype(str)

In [9]:
database_data

,Medicine_Name_DB,Quantity_DB,Dosage_DB,Database_Rates
0,Avastin,100.0,Injection,33863
1,Actorise,40.0,Injection,2928.2
2,Actorise,25.0,Injection,1815
3,Actorise,60.0,Injection,4235
4,Azel,80.0,Capsule,8700
...,...,...,...,...
187040,Zopen,40.0,Tablet,54
187041,Zenprega D,3.0,Tablet,302
187042,Zonpaar D,30.0,Capsule,99
187043,Zeof,0.3,,20


In [22]:
database_data['Medicine_Name_DB'] = database_data['Medicine_Name_DB'].str.title()

In [23]:
bills_data['Medicine_Name_Bill'] = bills_data['Medicine_Name_Bill'].str.title()

In [19]:
# Convert columns to strings where necessary
bills_data['Quantity_Bill'] = bills_data['Quantity_Bill'].fillna('0').astype(str)
bills_data['Dosage_Bill'] = bills_data['Dosage_Bill'].fillna('').astype(str)
bills_data['Medicine_Name_Bill'] = bills_data['Medicine_Name_Bill'].fillna('').astype(str)

database_data['Quantity_DB'] = database_data['Quantity_DB'].astype(str)
database_data['Dosage_DB'] = database_data['Dosage_DB'].astype(str)
database_data['Medicine_Name_DB'] = database_data['Medicine_Name_DB'].astype(str)

print("Cleaned Bills Data:")
print(bills_data)
print("Cleaned Database Data:")
print(database_data)

Cleaned Bills Data:
                Medicine_Name_Bill Quantity_Bill Dosage_Bill  Bill_Rates
0                  A To Z Ns Syrup             0                   190.0
1                        Abcixirel            10   Injection      8910.0
2                        Abcixirel            10   Injection      8910.0
3     Abd Spdnge X-Ray Detection (           500                   180.0
4                       Accutaz Hs           500                   143.0
...                            ...           ...         ...         ...
1273                                           0                     NaN
1274                                           0   Injection         NaN
1275                                           0   Injection         NaN
1276                                           0   Injection         NaN
1277                                           0   Injection         NaN

[1278 rows x 4 columns]
Cleaned Database Data:
       Medicine_Name_DB Quantity_DB  Dosage_DB Database_

In [24]:
# Convert columns to string where necessary and handle NaN values
bills_data['Quantity_Bill'] = bills_data['Quantity_Bill'].fillna('0').astype(str)
bills_data['Dosage_Bill'] = bills_data['Dosage_Bill'].fillna('').astype(str)
bills_data['Medicine_Name_Bill'] = bills_data['Medicine_Name_Bill'].fillna('').astype(str)

database_data['Quantity_DB'] = database_data['Quantity_DB'].fillna('0').astype(str)
database_data['Dosage_DB'] = database_data['Dosage_DB'].fillna('').astype(str)
database_data['Medicine_Name_DB'] = database_data['Medicine_Name_DB'].fillna('').astype(str)

# Remove leading/trailing whitespace
bills_data['Medicine_Name_Bill'] = bills_data['Medicine_Name_Bill'].str.strip()
bills_data['Dosage_Bill'] = bills_data['Dosage_Bill'].str.strip()
database_data['Medicine_Name_DB'] = database_data['Medicine_Name_DB'].str.strip()
database_data['Dosage_DB'] = database_data['Dosage_DB'].str.strip()

# Function to filter database_data based on exact match
def filter_database_exact(bills_row):
    return database_data[
        (database_data['Medicine_Name_DB'] == bills_row['Medicine_Name_Bill']) &
        (database_data['Quantity_DB'] == bills_row['Quantity_Bill']) &
        (database_data['Dosage_DB'] == bills_row['Dosage_Bill'])
    ]

# Apply the filtering function to each row of bills_data
filtered_results = pd.concat([filter_database_exact(row) for _, row in bills_data.iterrows()])

# Drop duplicate rows in case of overlaps
filtered_results = filtered_results.drop_duplicates()

# Optionally, you might want to select specific columns
filtered_results = filtered_results[['Medicine_Name_DB', 'Quantity_DB', 'Dosage_DB', 'Database_Rates']]

# Print or save the filtered data
print("Final filtered results:")
print(filtered_results)


Final filtered results:
            Medicine_Name_DB Quantity_DB Dosage_DB Database_Rates
10                    Alprax        0.25    Tablet             35
9973                Budecort         0.5                   133.25
54383   Gentalene Plus Cream           0                      138
55667   Gentalene Plus Cream           0                       74
147242       Thrombophob Gel           0                    207.6


In [27]:
from fuzzywuzzy import fuzz
import pandas as pd

# Function to filter database_data based on approximate match
def filter_database_approx(bills_row):
    return database_data[
        (database_data['Medicine_Name_DB'].apply(lambda x: fuzz.ratio(x, bills_row['Medicine_Name_Bill'])) >= 90) &
        (database_data['Dosage_DB'].apply(lambda x: fuzz.ratio(x, bills_row['Dos        (database_data['Quantity_DB'].apply(lambda x: abs(int(x) - int(bills_row['Quantity_Bill'])) <= 1)) &  # Tolerate minor differences in quantity
age_Bill'])) >= 90)
    ]

# Apply the filtering function to each row of bills_data
filtered_results = pd.concat([filter_database_approx(row) for _, row in bills_data.iterrows()])

# Drop duplicate rows in case of overlaps
filtered_results = filtered_results.drop_duplicates()

# Optionally, you might want to select specific columns
filtered_results = filtered_results[['Medicine_Name_DB', 'Quantity_DB', 'Dosage_DB', 'Database_Rates']]

# Print or save the filtered data
print("Final filtered results:")
print(filtered_results)


ValueError: invalid literal for int() with base 10: '100.0'

In [29]:
from fuzzywuzzy import fuzz
import pandas as pd

# Convert 'Quantity_DB' to numeric, handling errors
database_data['Quantity_DB'] = pd.to_numeric(database_data['Quantity_DB'], errors='coerce')
bills_data['Quantity_Bill'] = pd.to_numeric(bills_data['Quantity_Bill'], errors='coerce')

# Function to filter database_data based on approximate match
def filter_database_approx(bills_row):
    return database_data[
        (database_data['Medicine_Name_DB'].apply(lambda x: fuzz.ratio(x, bills_row['Medicine_Name_Bill'])) >= 90) &
        (database_data['Quantity_DB'].apply(lambda x: abs(x - bills_row['Quantity_Bill']) <= 1)) &  # Tolerate minor differences in quantity
        (database_data['Dosage_DB'].apply(lambda x: fuzz.ratio(x, bills_row['Dosage_Bill'])) >= 90)
    ]

# Apply the filtering function to each row of bills_data
filtered_results = pd.concat([filter_database_approx(row) for _, row in bills_data.iterrows()])

# Drop duplicate rows in case of overlaps
filtered_results = filtered_results.drop_duplicates()

# Optionally, you might want to select specific columns
filtered_results = filtered_results[['Medicine_Name_DB', 'Quantity_DB', 'Dosage_DB', 'Database_Rates']]

# Print or save the filtered data
print("Final filtered results:")
print(filtered_results)


Final filtered results:
       Medicine_Name_DB  Quantity_DB  Dosage_DB Database_Rates
2735          Abcixirel        10.00  Injection           8100
10               Alprax         0.25     Tablet             35
42               Alprax         0.00     Tablet           73.5
902              Alprax         1.00     Tablet         138.15
2346             Alprax         1.00     Tablet           51.1
...                 ...          ...        ...            ...
147242  Thrombophob Gel         0.00                     207.6
161054           Veltam         0.00     Tablet            206
161151           Veltam         0.00     Tablet             79
161087       Voveran Sr        75.00     Tablet          109.5
177541           Zostum         1.00  Injection            499

[90 rows x 4 columns]


In [31]:
filtered_results.head(50)

,Medicine_Name_DB,Quantity_DB,Dosage_DB,Database_Rates
2735,Abcixirel,10.00,Injection,8100
10,Alprax,0.25,Tablet,35
42,Alprax,0.00,Tablet,73.5
902,Alprax,1.00,Tablet,138.15
2346,Alprax,1.00,Tablet,51.1
3382,Alprax,1.00,Tablet,30.8
3949,Alpra,0.00,Tablet,18.9
5736,Alpra,0.20,Tablet,9.5
1457,Amikacin,250.00,Injection,38.7
4746,Amikamac,250.00,Injection,55.5


In [1]:
def Download(url, username, password):
    r = requests.get('https://www.netflix.com/' + url, stream=True, auth=(username, password))
    f = open('Output.mp4', 'wb')

    for chunk in r.iter_content(chunk_size=255):
        if chunk: # filter out keep-alive new chunks
            f.write(chunk)

# os.system(f"curl https://www.netflix.com/{url} -u '{username}:{password}' --output ~/Documents/Programming/Netflix_Downloader/Output.mp4")